In [1]:
import os
import pandas as pd
from dataclasses import dataclass
from pathlib import Path


C:\Users\roshi\AppData\Local\Temp\ipykernel_12708\616916744.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
os.chdir("../")
%pwd

'c:\\Users\\roshi\\OneDrive\\Desktop\\Git-1\\Wine_Quality_ML'

In [4]:
from src.Wine_Quality_ML.utils import *
from src.Wine_Quality_ML.constants import *

In [3]:
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metrics_file_path: Path
    all_params: dict
    target_column: str


In [6]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = Config_File_Path,
            params_path = Params_File_Path,
            schema_path = Schema_File_Path
        ):
        
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        self.schema = read_yaml(schema_path)

        create_directories([self.config.artifacts_root])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.Target_Column
        
        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir= config.root_dir,
            test_data_path= config.test_data_path,
            model_path= config.model_path,
            metrics_file_path= config.metrics_file_path,
            all_params= params,
            target_column= schema.name
        )

        return model_evaluation_config

In [12]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import joblib
from urllib.parse import urlparse
import numpy as np

In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    def metrics_eval(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        (rmse, mae, r2) = self.metrics_eval(test_y, predicted_qualities)
        
        # Saving metrics as local
        scores = {"rmse": rmse, "mae": mae, "r2": r2}
        save_json(path= Path(self.config.metrics_file_path), data=scores)

In [17]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    model_eval = ModelEvaluation(config= model_eval_config)
    model_eval.save_results()
except Exception as e:
    raise e

TypeError: save_json() got an unexpected keyword argument 'path'